# Solutions to the micrograd exercises


## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  _a = -3*(a**2) - (0.5)*(a**-0.5)
  _b = 3*cos(3*b) + 2.5*(b**1.5)
  _c = (c**-2.0)
  return [_a, _b, _c] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus
# you should not call the function df from the last cell

# -----------
def n_grad(f, a, b ,c, h=1e-6):
    grad = [0, 0, 0]
    grad[0] = (f(a+h, b, c)-f(a, b, c ))/h
    grad[1] = (f(a, b+h, c)-f(a, b, c ))/h
    grad[2] = (f(a, b, c+h)-f(a, b, c ))/h
    return grad

numerical_grad = n_grad(f, 2, 3, 4)
# -----------


for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------

def n_grad2(f, a, b ,c, h=1e-6):
    grad = [0, 0, 0]
    grad[0] = (f(a + h, b, c) - f(a - h, b, c)) / (2 * h)
    grad[1] = (f(a, b + h, c) - f(a, b - h, c)) / (2 * h)
    grad[2] = (f(a, b, c + h) - f(a, b, c - h)) / (2 * h)
    return grad

numerical_grad2 = n_grad2(f, 2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [20]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '+')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out


  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data ** other, (self,), f'**{other}')

    def _backward():
      self.grad += (other * (self.data ** (other - 1))) * out.grad
    out._backward = _backward

    return out


  def exp(self):
    e = exp(self.data)

    out = Value(e, (self,), 'exp')

    def _backward():
        self.grad += e * out.grad
    out._backward = _backward

    return out


  def log(self):
    a = self.data

    out = Value(log(a), (self,), 'log')

    def _backward():
        self.grad += (1/a) * out.grad
    out._backward = _backward

    return out


  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

  def __radd__(self, other):
    return self + other

  def __rmul__(self, other):
    return self * other

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def __rsub__(self, other):
    return other + (-self)

  def __truediv__(self, other):
    return self * (other**-1)

  def __rtruediv__(self, other):
    return other * (self**-1)

  def __repr__(self):
    return f"Value(data={self.data}, grad={self.grad})"

In [22]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [29]:
# verify the gradient using the torch library
# torch should give you the exact same gradient

# NOTE: The gradients are stored in logits.grad, which is a tensor.

import torch

# this is the negative log likelihood loss function, pervasive in classification
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], dtype=torch.double)
logits.requires_grad=True
probs = torch.nn.functional.softmax(logits, dim=-1)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.item())

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits.grad[dim].item()}")


2.1755153626167143
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400987


In [25]:
import torch

# Define the logits as a tensor with requires_grad=True to enable gradient computation
logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)

# Define the softmax function
def softmax_torch(logits):
    exp_logits = torch.exp(logits)
    return exp_logits / torch.sum(exp_logits)

# Compute probabilities using the softmax function
probs_torch = softmax_torch(logits_torch)

# Compute the negative log likelihood loss
loss_torch = -torch.log(probs_torch[3])

# Backpropagate to compute gradients
loss_torch.backward()

# Print the loss and gradients
print(f"Loss: {loss_torch.item()}")
for dim in range(4):
    print(f"Gradient for dim {dim}: {logits_torch.grad[dim].item()}")

# Expected answer
ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]

# Check the results
for dim in range(4):
    ok = 'OK' if abs(logits_torch.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, torch returns {logits_torch.grad[dim].item()}")

Loss: 2.1755154132843018
Gradient for dim 0: 0.041772566735744476
Gradient for dim 1: 0.8390244245529175
Gradient for dim 2: 0.005653302185237408
Gradient for dim 3: -0.8864503502845764
OK for dim 0: expected 0.041772570515350445, torch returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, torch returns 0.8390244245529175
OK for dim 2: expected 0.005653302662216329, torch returns 0.005653302185237408
OK for dim 3: expected -0.8864503806400986, torch returns -0.8864503502845764
